## Notebook cleaned for refactoring

* 
* Kept data cleaning and feature engineering, but 
* Removed all machine learning, sklearn and visualisiation aspects 


In [4]:
# Basic imports
import pandas as pd
import numpy as np

In [5]:
# Loading of the dataset via pandas
kc_data = pd.read_csv("data/King_County_House_prices_dataset.csv")

### Data cleaning


The .info() output is useful for writing the data validation model, so let's keep it in here for the time being 

In [6]:
kc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [36]:
# ## REF
# from pydantic import BaseModel, int

# class House(BaseModel):
#     id: int
#     date: # Date stuff
#     price: float
#     bedrooms: int
#     sqft_living: int
#     sqft_lot: int
#     floors: float
#     waterfront: int # change to integer

ImportError: cannot import name 'BaseClass' from 'pydantic' (/Users/bromethius/neuefische/refactoring-pipeline-project/.venv/lib/python3.11/site-packages/pydantic/__init__.py)

**Extreme values**

* Here the data scientist removed extreme buildings with bedrooms >> bathrooms
* Suggest a rule to cope with other extreme properties:
    * bedrooms / bathrooms > 10

In [10]:
# Output of the line in which the condition "bedrooms = 33" is present.
# kc_data.query('bedrooms == 33')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,0.0,0.0,...,7,1040,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700


In [11]:
# kc_data.drop(15856, axis=0, inplace=True)

In [22]:
## REF: Make the requirements a bit stricter, 5 bedrooms per bathroom is too few bathrooms
## More than 2 bathrooms per bedroom is too many

from sklearn.base import BaseEstimator, TransformerMixin

class BathBedTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        ratio = X.bedrooms / X.bathrooms
        idx = X.index[(ratio >= 5. ) | (ratio < 0.5 )]

        X.drop(index=idx, inplace=True)

        return X


In [25]:
bathbedratio_transformed = BathBedTransformer()
kc_data_cleaned = bathbedratio_transformed.fit_transform(kc_data)

In [26]:
kc_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21536 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21536 non-null  int64  
 1   date           21536 non-null  object 
 2   price          21536 non-null  float64
 3   bedrooms       21536 non-null  int64  
 4   bathrooms      21536 non-null  float64
 5   sqft_living    21536 non-null  int64  
 6   sqft_lot       21536 non-null  int64  
 7   floors         21536 non-null  float64
 8   waterfront     19170 non-null  float64
 9   view           21473 non-null  float64
 10  condition      21536 non-null  int64  
 11  grade          21536 non-null  int64  
 12  sqft_above     21536 non-null  int64  
 13  sqft_basement  21536 non-null  object 
 14  yr_built       21536 non-null  int64  
 15  yr_renovated   17705 non-null  float64
 16  zipcode        21536 non-null  int64  
 17  lat            21536 non-null  float64
 18  long       

**Remove "?" in "sqft_basement**


* Notice the DS replaced first cleaned "sqft_basement", THEN replaced it by imputing from two other features
* Which approach is robuster? Check and just do one or the other

In [18]:
# We replace "?" with Nan
kc_data['sqft_basement'] = kc_data['sqft_basement'].replace('?', np.NaN)
# And we change the dtype of the column "sqft_basement" to float
kc_data['sqft_basement'] = kc_data['sqft_basement'].astype(float)

# RFT: Now calling this imputing "sqft_basement2"
kc_data.eval('sqft_basement2 = sqft_living - sqft_above', inplace=True)

In [19]:
# RFT: Check missings from both approaches 
print(f'Cleaning and type conversion: {round(kc_data.sqft_basement.isna().mean() * 100,2)} % missing')
print(f'Imputing: {round(kc_data.sqft_basement2.isna().mean() * 100,2)} % missing')


Cleaning and type conversion: 2.1 % missing
Imputing: 0.0 % missing


So imputing seems more successful. Are the data fundamentally different?

In [27]:
kc_data[['sqft_basement2', 'sqft_basement']]

,sqft_basement2,sqft_basement
0,0,0.0
1,400,400.0
2,0,0.0
3,910,910.0
4,0,0.0
...,...,...
21592,0,0.0
21593,0,0.0
21594,0,0.0
21595,0,0.0


In [32]:
print("Mean square percentage error of imputed sqft_basement2 versus cleaned sqft_basement")
np.sum(np.sqrt((kc_data.sqft_basement2 - kc_data.sqft_basement)**2) ) / kc_data.sqft_basement.sum() * 100

Mean square percentage error of imputed sqft_basement2 versus cleaned sqft_basement


0.0

OK, So we will skip the Data scientist's cleaning step for sqft_basement and impute as follows:

In [33]:
kc_data.eval('sqft_basement = sqft_living - sqft_above', inplace=True)

In [31]:
# RFT: Imputing is more successful than cleaning
# RFT: Create imputer for the basement sqft area

class BasementAreaTransformer(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.copy()
        X.sqft_basement = X.sqft_living - X.sqft_above
        return X

#RFT:
basement_trans = BasementAreaTransformer()
kc_data_cleaned = basement_trans.fit_transform(kc_data_cleaned)
round(kc_data_cleaned.isna().mean() * 100,2)

id                0.00
date              0.00
price             0.00
bedrooms          0.00
bathrooms         0.00
sqft_living       0.00
sqft_lot          0.00
floors            0.00
waterfront       10.99
view              0.29
condition         0.00
grade             0.00
sqft_above        0.00
sqft_basement     0.00
yr_built          0.00
yr_renovated     17.79
zipcode           0.00
lat               0.00
long              0.00
sqft_living15     0.00
sqft_lot15        0.00
dtype: float64

**Missings Values**
Summary:
* Fillna in view and waterfront with 0
* Calculate last known change from yr_renovated and yr_built, preferably with a simpler and more efficient, parallelizable function



In [14]:
# We replace Nan values in "view" with the most frequent expression (0)
kc_data['view'].fillna(0, inplace=True)

In [16]:
# We replace Nan values in "waterfront" with the most frequent expression (0)
kc_data.waterfront.fillna(0, inplace=True)

In [37]:
kc_data.waterfront.mode().values[0]

0.0

In [38]:
#REF
class ModalImputer(BaseEstimator,TransformerMixin):
    def __init__(self,columns=['view','waterfront']):
        self._cols = columns
        self._modes = dict()
    
    def fit(self, X, y=None):
        for col in self._cols:
            self._modes[col] = X[col].mode().values[0]
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        for col in self._cols:
            X[col].fillna(self._modes[col], inplace=True)
        
        return X
    
#REF: test
modal_imputer = ModalImputer()
kc_data_cleaned = modal_imputer.fit_transform(kc_data_cleaned)



In [39]:
round(kc_data_cleaned.isna().mean() * 100,2)

id                0.00
date              0.00
price             0.00
bedrooms          0.00
bathrooms         0.00
sqft_living       0.00
sqft_lot          0.00
floors            0.00
waterfront        0.00
view              0.00
condition         0.00
grade             0.00
sqft_above        0.00
sqft_basement     0.00
yr_built          0.00
yr_renovated     17.79
zipcode           0.00
lat               0.00
long              0.00
sqft_living15     0.00
sqft_lot15        0.00
dtype: float64

In [55]:
mask = (kc_data_cleaned.yr_renovated.isna()) | (kc_data_cleaned.yr_renovated==0)
kc_data_cleaned.yr_renovated.loc[~mask]

kc_data_cleaned.yr_renovated.where(~mask, kc_data_cleaned.yr_built)

0        1955.0
1        1991.0
2        1933.0
3        1965.0
4        1987.0
          ...  
21592    2009.0
21593    2014.0
21594    2009.0
21595    2004.0
21596    2008.0
Name: yr_renovated, Length: 21536, dtype: float64

In [57]:
# REF
class LastChangeTransformer(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.copy()
        mask = (kc_data_cleaned.yr_renovated.isna()) | (kc_data_cleaned.yr_renovated==0)
        new_col = kc_data_cleaned.yr_renovated.where(
            ~mask, kc_data_cleaned.yr_built).astype(int)
        X['last_known_change'] = new_col
        X.drop(['yr_renovated','yr_built'], axis=1, inplace=True)

        return X


In [59]:
   
last_change_transformer = LastChangeTransformer()
kc_data_cleaned_2 = last_change_transformer.fit_transform(kc_data_cleaned)

kc_data_cleaned_2.last_known_change

0        1955
1        1991
2        1933
3        1965
4        1987
         ... 
21592    2009
21593    2014
21594    2009
21595    2004
21596    2008
Name: last_known_change, Length: 21536, dtype: int64

In [19]:
# We will create an empty list in which we will store values
last_known_change = []

# For each row in our data frame, we look at what is in the column "yr_renovated".
for idx, yr_re in kc_data.yr_renovated.items():
    # if "yr_renovated" is 0 or contains no value, we store the year of construction of the house in our empty listes ab
    if str(yr_re) == 'nan' or yr_re == 0.0:
        last_known_change.append(kc_data.yr_built[idx])
    # if there is a value other than 0 in the column "yr_renovated", we transfer this value into our new list
    else:
        last_known_change.append(int(yr_re))

# We create a new column and take over the values of our previously created list
kc_data['last_known_change'] = last_known_change

# We delete the "yr_renovated" and "yr_built" columns
kc_data.drop("yr_renovated", axis=1, inplace=True)
kc_data.drop("yr_built", axis=1, inplace=True)

### Exploratory Data Analysis

* This section has been deleted because it is not relevant for the refactoring

### Feature Engineering

* sqft_price is feature engineered but then later dropped, we delete this code
* center_distance is calculated
* distance from waterfront is calculated, although this is quite complicated code, we could use geopandas for more efficient sjoin functionalities
* Two unnecessary variables are created: 'delta_lat', 'delta_long', which will be excluded from the refactoring
* Some feature engineering crept into the modelling part, but this involved raising all variables to the power of 2. We will not store these features in the database because they are easily computed after the data pipeline. 

In [ ]:
# Absolute difference of latitude between centre and property
kc_data['delta_lat'] = np.absolute(47.62774- kc_data['lat'])
# Absolute difference of longitude between centre and property
kc_data['delta_long'] = np.absolute(-122.24194-kc_data['long'])
# Distance between centre and property
kc_data['center_distance']= ((kc_data['delta_long']* np.cos(np.radians(47.6219)))**2 
                                   + kc_data['delta_lat']**2)**(1/2)*2*np.pi*6378/360


**Distance to the beach promenade**

In [ ]:
# This function helps us to calculate the distance between the house overlooking the seafront and the other houses.
def dist(long, lat, ref_long, ref_lat):
    '''dist computes the distance in km to a reference location. Input: long and lat of 
    the location of interest and ref_long and ref_lat as the long and lat of the reference location'''
    delta_long = long - ref_long
    delta_lat = lat - ref_lat
    delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
    return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

In [ ]:
# All houses with "waterfront" are added to the list
water_list= kc_data.query('waterfront == 1')
water_list.head()

In [ ]:
water_distance = []
# For each row in our data frame we now calculate the distance to the seafront
for idx, lat in kc_data.lat.iteritems():
    ref_list = []
    for x,y in zip(list(water_list.long), list(water_list.lat)):
        ref_list.append(dist(kc_data.long[idx], kc_data.lat[idx],x,y).min())
    water_distance.append(min(ref_list))

In [ ]:
# wir erstellen eine neue Spalte und übernehmen die Werte unserer vorher erstellten Liste
kc_data['water_distance'] = water_distance

In [ ]:
## REF: Create center distance with geopandas

from geopandas import 
from geopandas.geometries import Point

class DistanceCenterTransformer(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        #Create geodataseries from X Y where lat=y and x=long
        gds =  
        center = Point([47.62774,-122.24194])
        #Calculate distance to center
        dist = gds.distance(center)
        X['center_distance'] = dist.astype(float)

        return X
    
class DistanceWaterfrontTransformer(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        self._waterfront = X.query('waterfront == 1')
        self._water_gds = # create geodataseries from xy
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        #Create geodataseries from X Y where lat=y and x=long
        gds =  ###
        #Sjoin gds with nearest waterfront, keeping only waterfront geometry
        closest_water = gds.sjoin###
        #Calculate distance between two points of te new column
        dist = 
        # Enter the new column
        X['water_distance'] = dist.astype(float)
        return X

In [ ]:
from pandas.testing import assert_series_equal
distance_center_transform = DistanceCenterTransformer()
distance_water_transform = DistanceWaterfrontTransformer()

kc_data_cleaned_2 = distance_center_transform.fit_transform(kc_data_cleaned_2)
kc_data_cleaned_2 = distance_water_transform.fit_transform(kc_data_cleaned_2)

assert_series_equal(kc_data_cleaned_2['water_distance'], kc_data['water_distance'])
assert_series_equal(kc_data_cleaned_2['center_distance'], kc_data['center_distance'])


### Modeling

* This part of the notebook has been deleted because it is not relevant for the data engineering refactoring
